<a href="https://colab.research.google.com/github/Zorug/Steam-Stuff/blob/master/BS_%26_CSV_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Info

comparação com os dados do site: https://www.steamcardexchange.net 

extração do csv: https://steam.tools/cards/

In [1]:
""" info da columns:
Game: Name of the set's game
# Cards: Number of cards in the set
Set Price: The total price of all the cards for the set
Card Avg: The averege price of a card in the set
Booster Avg: The average price of a card from the set's booster
Booster %: How many percent cheaper avarage booster cards are for the set
Emote Avg: The average price of the set's emoticons
BG Avg: The average price of the set's bachgrounds
Avg Qty: The average quantity of items on the market
Discount: The money you would make by selling all your drops for that game
Added: The date the set was added
"""

" info da columns:\nGame: Name of the set's game\n# Cards: Number of cards in the set\nSet Price: The total price of all the cards for the set\nCard Avg: The averege price of a card in the set\nBooster Avg: The average price of a card from the set's booster\nBooster %: How many percent cheaper avarage booster cards are for the set\nEmote Avg: The average price of the set's emoticons\nBG Avg: The average price of the set's bachgrounds\nAvg Qty: The average quantity of items on the market\nDiscount: The money you would make by selling all your drops for that game\nAdded: The date the set was added\n"

# Importanto libs e dados

In [2]:
import pandas as pd

#csv_source = 'STC_set_data.csv'
csv_source = 'https://raw.githubusercontent.com/Zorug/Steam-Stuff/master/STC_set_data.csv'
dados_csv = pd.read_csv(csv_source)
dados_csv.shape

(9156, 16)

In [3]:
dados_csv.head(2)

,Game,# Owned,# Unique,# Cards,Badge Lvl,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Emote Avg,BG Avg,Avg Qty,Discount,Added,AppId
0,City Game Studio,0,0,12,0,1.25,1.25,0.10,NaN,0.0,0.00,0.00,17,0.52,2019-10-24,726840
1,Transport Services,0,0,5,0,1.85,1.85,0.37,NaN,0.0,0.00,0.00,4,0.97,2019-10-23,993690


#Organizando DataSet

## Removendo dados inúteis para o teste

In [0]:
dados_csv = dados_csv.drop(['# Owned', '# Unique', 'Badge Lvl','Emote Avg', 'BG Avg', 'Discount', 'Added'], axis = 1)

In [5]:
dados_csv.sample(2)

,Game,# Cards,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Avg Qty,AppId
4850,Aircraft War X,5,0.26,0.26,0.05,0.03,40.0,296,508670
7721,Drew and the Floating Labyrinth,5,0.42,0.42,0.08,0.05,37.5,30,346200


##Reorganizando os tipos

In [6]:
dados_csv.dtypes

Game            object
# Cards          int64
Set Price       object
Price Diff      object
Card Avg       float64
Booster Avg    float64
Booster %       object
Avg Qty          int64
AppId            int64
dtype: object

In [0]:
def remove_virgula_perdida(sentenca): # aqui estamos esvitando um erro de conversão
    sentenca = sentenca.replace(',','')
    return sentenca

In [0]:
para_float = ['Set Price', 'Price Diff', 'Booster %']
for coluna in dados_csv[para_float]:
    dados_csv[coluna] = dados_csv[coluna].apply(remove_virgula_perdida)

dados_csv[para_float] = dados_csv[para_float].astype('float64')
dados_csv['AppId'] = dados_csv['AppId'].astype('str')

In [9]:
dados_csv.dtypes

Game            object
# Cards          int64
Set Price      float64
Price Diff     float64
Card Avg       float64
Booster Avg    float64
Booster %      float64
Avg Qty          int64
AppId           object
dtype: object

# BeautifulSoup

In [0]:
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [11]:
amostra = dados_csv.sample(1)
amostra

,Game,# Cards,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Avg Qty,AppId
4391,Detective Hank and the Golden Sneeze,6,0.88,0.88,0.15,NaN,0.0,16,503980


In [0]:
#qnd_cartas = amostra['# Cards'].values[0]
#qnd_cartas

In [13]:
AppId = amostra['AppId'].values[0]
AppId

'503980'

In [14]:
url = 'https://www.steamcardexchange.net/index.php?gamepage-appid-'+AppId
url

'https://www.steamcardexchange.net/index.php?gamepage-appid-503980'

In [0]:
uClient = urlopen(url)
page_html = uClient.read()
uClient.close()

In [0]:
soup = BeautifulSoup(page_html, 'html.parser')

In [20]:
cards_script = soup.find("div",{"id":"cards"})
card_list = cards_script.findAll("div",{"class":"showcase-element"})
temporario = []
for item in card_list: # retira itens vazios
  if not item.a == None: temporario.append(item)
card_list = temporario

precos_das_cartas = []
for item in card_list: 
  try: precos_das_cartas.append( ((item.find("div", {"class":"element-button"})).text)[8:] )
  except:
    print('Arumando erro 368020, sem valor para carta... adicinonando NaN')
    precos_das_cartas.append( 'nan' )

precos_das_cartas = pd.Series(precos_das_cartas).apply(remove_virgula_perdida).astype('float64')
precos_das_cartas

0    0.07
1    0.15
2    0.18
3    0.07
4    0.27
5    0.14
dtype: float64